## 0. Setup
*Import model, analysis module, logging. All original code using open-source tools.*

In [1]:
from models_config import stanford_model as model
from log_analysis import first_pass, second_pass

%reload_ext autoreload
%autoreload 2
%matplotlib inline

Fetching 7 files:   0%|          | 0/7 [00:00<?, ?it/s]

Device set to use cpu


## 1. OCR
*Can handle PDF, images, dicom. Uses Tesseract, pydicom, and pdf2image libraries. OCR is a key component in the pipeline as a text model cannot work well on text that is not included in the input. So the OCR must be highly accurate and versatile, being able to handle noisy images and structured data (like tables).*

In [2]:
!python tesseract_test.py data/sample_pdf.pdf

Saved JSON to ocr_output/sample_pdf/sample_pdf_page1_ocr.json and text to ocr_output/sample_pdf/sample_pdf_page1.txt
Saved JSON to ocr_output/sample_pdf/sample_pdf_page2_ocr.json and text to ocr_output/sample_pdf/sample_pdf_page2.txt
Saved JSON to ocr_output/sample_pdf/sample_pdf_page3_ocr.json and text to ocr_output/sample_pdf/sample_pdf_page3.txt
Saved JSON to ocr_output/sample_pdf/sample_pdf_page4_ocr.json and text to ocr_output/sample_pdf/sample_pdf_page4.txt
Saved JSON to ocr_output/sample_pdf/sample_pdf_page5_ocr.json and text to ocr_output/sample_pdf/sample_pdf_page5.txt
Saved JSON to ocr_output/sample_pdf/sample_pdf_page6_ocr.json and text to ocr_output/sample_pdf/sample_pdf_page6.txt
Outputs saved in ocr_output/sample_pdf


In [3]:
text = ""
for i in range(1, 7):
    with open(f"ocr_output/sample_pdf/sample_pdf_page{i}.txt", "r") as f:
        page_text = f.read()
        text += page_text + "\n"


print(text)

UNIVERSITY MEDICAL CTR
Department: Neurology / Internal Med
Document Type: ED NOTE + CONSULT + DISCHARGE
Generated: 2024.03.22 07:14 AM
Printed by: J.Nguyen (unit clerk) ext 4021
PATIENT INFORMATION
Name: Jenniferr K. Lee
AKA: Jenny Lee/ J. Lee
DOB: 5-6-94
Age: 29 y/o
Sex: F
Preferred Language: English
Race/ethnicity (self-reported): Korean-American
MRN: 00077219
Acct#: A-77821
Encounter ID: 44-92-77 1
Home Address:
455 San Mateo Ave
Apt# 3B
Redwood City, CA 94063
CURRENT ADDRESS (per patient, moved recently):
4127 W. Elmst Apt #3B
Springfeld, IL 62704
Phone: 312-555-7712
Alt/cell: (312) 555 77 13
Email: jlee94 @ yahoo.com
Emergency contact:
Marry Smith (spouse) 217.555.0198
relationship: wife
Secondary contact:
Anne McKinly (sister) 773.555.8891
Primary care provider (outside):
Dr. S. Patel, MD
Northside Pulmonology
Fax: 847 555 4021
CHIEF COMPLAINT
"migraine flare" + dizziness + chest tightness intermittently
HISTORY OF PRESENT ILLNESS
Pt is a 29 yo F presenting on 03/21/2024 with he

## 2. PII Detection
*We chose to use Microsoft Presidio and Stanford Research's Clinical DEID transformer model. We felt these tools best matched the resources John Snow Labs provided with similar accuracy and greater accessibility. When using our pipeline, the user has the ability look over the file to make sure all necessary PHI is removed. We felt that possibility of noisy images leading to false negatives posed too high of a risk to leave the human out of the loop.*

# First Pass
*The first pass through the model attempts to tag every PII entity. If the accuracy is perfect, nice. If not, the user has a chance to make manual changes.*

In [4]:
first_pass_result, tagged_p1, pass_idx = first_pass(model, text, doc_id=1, case="sample")
print(first_pass_result, tagged_p1)

{}
<LOCATION>
Department: Neurology / Internal Med
Document Type: ED NOTE + CONSULT + DISCHARGE
Generated: <DATE_TIME> 07:14 AM
Printed by: <PERSON> (unit clerk) ext <PHONE_NUMBER>
PATIENT INFORMATION
Name: <PERSON>
AKA: <PERSON>
DOB: <DATE_TIME>
<AGE>
Sex: F
Preferred Language: English
Race/ethnicity (self-reported): Korean-American
<MRN>
Acct#: <ID>
Encounter ID: <ID>
Home Address:
<LOCATION>
Apt# 3B
<LOCATION> <ZIPCODE>
CURRENT ADDRESS (per patient, moved recently):
<LOCATION> Apt #3B
<LOCATION> <PHONE_NUMBER>
Phone: <PHONE_NUMBER>
Alt/cell: <PHONE_NUMBER>
Email: jlee94 @ <ORGANIZATION>
Emergency contact:
<PERSON> (spouse) <PHONE_NUMBER>
relationship: wife
Secondary contact:
<PERSON> (sister) <PHONE_NUMBER>
Primary care provider (outside):
<PERSON>
<LOCATION> Pulmonology
Fax: <PHONE_NUMBER>
CHIEF COMPLAINT
"migraine flare" + dizziness + chest tightness intermittently
HISTORY OF PRESENT ILLNESS
Pt is a <AGE> F presenting on <DATE_TIME> with headache for 2 days.
Pain begins behind R e

/opt/miniconda3/envs/deid-pipeline/lib/python3.11/site-packages/spacy_huggingface_pipelines/token_classification.py:129: UserWarning: Skipping annotation, {'entity_group': 'PATIENT', 'score': np.float32(0.5035428), 'word': '.', 'start': 522, 'end': 523} is overlapping or can't be aligned for doc 'UNIVERSITY MEDICAL CTR
Department: Neurology / Internal Med
Document Type: ED NOTE + CONSULT + DISCH...'
  warnings.warn(


# Second Pass
*The user can not identify the entities the model missed (false negatives), and the entities that the model should not have tagged (false positives).*

In [5]:
deny_list = ["jlee94", "07:14 AM", "22:10"]
allow_list = ["29 y/o", "29 yo"]

second_pass_results, tagged_p2, pass_idx = second_pass(model, text, case="sample", doc_id=pass_idx, allow_list=allow_list, deny_list=deny_list)

print(second_pass_results)

[{'J. Nguyen PA-C', 'J.Nguyen'}, {'Dr. S. Patel, MD', 'S. Patel, MD', 'Dr. S. Patel'}, {'J. Lee', 'Jenniferr K. Lee', 'Jenny Lee/'}, {'Marry Smith'}, {'Rebecca Wong MD'}, {'Rosa Ramirez'}, {'Anne McKinly'}]
{'217.555.0198': '711.217.1233', 'yahoo.com': '*', 'jlee94': '*', '847 555 4021': '689.328.4266', 'S. Patel, MD': 'Jamie', 'Northside': 'East Jeffreyport', 'Dr. S. Patel': 'Jamie', '3-22-24': '12-22-24', 'FMLA': 'Daviston', '312.555.7766': '386.762.7237', 'South Loop Immediate': 'Davidview', 'J. Nguyen PA-C': 'Parker', 'Rebecca Wong MD': 'Rowan', '03/21/24': '12/21/24', '2024-02-17': '2024-11-18', '2024-03-22': '2024-12-22', '22:10': '*', '2024.03.21': '03-28-79', 'L-998211': '*', 'Anne McKinly': 'Jackie', '773.555.4410': '301.218.3190', 'Rosa Ramirez': 'Riley', 'IL': 'Carolynton', 'BlueCross of': '*', 'warehouse': '*', 'McArthuer Shipping Co.': '*', '2010': '01-20-92', '1990s': '08-09-42', '2/28/24': '11/29/24', 'Redwood City CA': 'West Carol', '1200 El Camino Real': 'New Heatherla

/opt/miniconda3/envs/deid-pipeline/lib/python3.11/site-packages/spacy_huggingface_pipelines/token_classification.py:129: UserWarning: Skipping annotation, {'entity_group': 'PATIENT', 'score': np.float32(0.5035428), 'word': '.', 'start': 522, 'end': 523} is overlapping or can't be aligned for doc 'UNIVERSITY MEDICAL CTR
Department: Neurology / Internal Med
Document Type: ED NOTE + CONSULT + DISCH...'
  warnings.warn(


## 3. Output Generation
*Now, we need to map the replacement strings back to the entities and burn them on to the original PDF. The burning works in a void, but we are currently working on the mapping part. The main roadblock right now is that the OCR chunks by individual word, while the model identifies entities that could be multiple words long. There are also aesthetic issues like font, font size, spacing, etc.*

In [6]:
from match_results import link_json
link_json("ocr_output/sample_pdf", "logs/sample/2/results_20260128_212023")

In [7]:
!python output_layout.py data/sample_pdf.pdf ocr_output/sample_pdf/

Saved to: deid_output/sample_pdf/sample_pdf_page1_deid.png
Saved to: deid_output/sample_pdf/sample_pdf_page2_deid.png
Saved to: deid_output/sample_pdf/sample_pdf_page3_deid.png
Saved to: deid_output/sample_pdf/sample_pdf_page4_deid.png
Saved to: deid_output/sample_pdf/sample_pdf_page5_deid.png
Saved to: deid_output/sample_pdf/sample_pdf_page6_deid.png
Saved PDF to: deid_output/sample_pdf/sample_pdf_deid.pdf
